In [ ]:
import requests
import grequests
import ipywidgets as widgets
from IPython.display import display
import json
import matplotlib.pyplot as plt
import numpy
from urllib3 import disable_warnings
from datetime import datetime, timedelta
import getpass
import traceback
from dateutil import parser as dateParser

In [ ]:
from urllib3 import disable_warnings
disable_warnings()

In [ ]:
import ipywidgets as widgets

class Data:
    def __init__(self, data, options=None):
        """
            Initializes the object
            
            Parameters
            ----------
            data: list or dict
                the data to be processed and aggregated
            options:  dict
                a dict of options
                
                "keys": list
                    a list of strings used to determine which properties of objects in arrays can be used to determine a key for that object
                    default: ["field", "names"]
                    
                "parse": list [str]
                    what types should be automatically parsed
                    default: ["int", "float", "date"]
                
            Returns
            -------
            Data
                The Data object
        """
        self.data = data
        self.paths = {}
        self.formattedData = None
        self.columns = {} # name:rule
        defaults = {
            "keys": ["field", "names"],
            "parse": ["int", "float","date"]
        }
        if isinstance(options, dict):
            for key, value in options.items():
                defaults[key] = value
        self.options = defaults
    
    def __str__(self):
        return ("Data [" + str(len(self.data)) + "]")
    
    @staticmethod
    def path2str(path):
        string = ""
        
        for loc in path:
            if isinstance(loc, str):
                string += "/" + loc
                
        return string
    
    def get_gui(self):
        """
            Returns a gui to allow the user to select data fields
            
            Returns
            -------
            ipywidget
                an ipywidget to be displayed
        """
        pathContainer = widgets.VBox()

        fullPathContainer = widgets.VBox()

        addButton = widgets.Button(description="Add")
        removeButton = widgets.Button(description="Remove")

        fullPathContainer.children = [pathContainer, addButton, removeButton]



        
        def getIndices(obj):
            def childrenToIndices(children):
                childrenDict = {}
                for index, child in enumerate(children):
                    childId = ""
                    try:
                        childId = "" + child["field"]
                    except:
                        try:
                            childId = "" + child["data_fields"]["name"]
                        except:
                            raise Error("children cannot be converted to dict")
                    childrenDict[childId] = [childId]

                return childrenDict

            def formatIfFinal(obj, index):
                try:
                    item = obj[index]
                except:
                    return index
                else:
                    if isinstance(item, list) or isinstance(item, dict):
                        return index + "/"
                    else:
                        return index

            indices = {}
            if isinstance(obj, dict):
                for index, value in obj.items():
                    # it's a dict
                    if isinstance(value, list):
                        try:
                            childrenIndices = childrenToIndices(value)
                        except:
                            indices[formatIfFinal(obj, index)] = [index]
                        else:
                            for childId, childIndex in childrenIndices.items():
                                indices[childId + "/"] = [index] + childIndex
                    else:
                        indices[formatIfFinal(obj, index)] = [index]

                return indices
            elif isinstance(obj, list):
                for count, child in enumerate(obj):
                    childIndices = getIndices(child)
                    for childFakeIndex, childIndex in childIndices.items():
                        try:
                            # sees if the index already exists
                            x = indices[childFakeIndex]
                        except:
                            indices[childFakeIndex] = [count - len(obj)] + childIndex
                return indices
            else:
                raise Exception("no more indices")

        def addPathWidget(b=None):
            def dataFromIndices(data, indices):
                #     print(indices)
                try:
                    currIndex = indices[0]
                except:
                    return [data]
                else:
                    if isinstance(data, list):
                        if isinstance(currIndex, int) and currIndex < 0:
                            out = []
                            for item in data:
                                try:
                                    out += dataFromIndices(item, indices[1:len(indices)])
                                except:
                                    pass
                            return out
                        elif isinstance(currIndex, str):
                            for item in data:
                                try:
                                    if item['field'] == currIndex:
                                        return dataFromIndices(item, indices[1:len(indices)])
                                except:
                                    try:
                                        if item['data_fields']['name'] == currIndex:
                                            return dataFromIndices(item, indices[1:len(indices)])
                                    except:
                                        pass
                    else:
                        try:
                            return dataFromIndices(data[currIndex], indices[1:len(indices)])
                        except:
                            pass
                    raise Exception("Index " + currIndex + " is invalid")

            path = []
            for i in range(len(pathContainer.children)):
                index = pathContainer.children[i].value
                p = index[1:len(index)]
                path += p
            try:
                data = dataFromIndices(self.data, path)
                indices = getIndices(data)
            except:
                traceback.print_exc()
                # it's final, add the path to something
                self.formattedData = None
                pathPretty = Data.path2str(path)
                print(pathPretty)
                self.paths[pathPretty] = path
                return
            else:
                try:
                    pathContainer.children[-1].disabled = True
                except:
                    pass
                finally:
                    for key, value in indices.items():
                        indices[key][0] = key
                    newDD = widgets.Dropdown(options=indices, disabled=False)
                    try:
                        newDD.value = indices['experiments/']
                    except:
                        pass
                    c = pathContainer.children
                    c += (newDD,)
                    pathContainer.children = c

        def removePathWidget(b=None):
            if (len(pathContainer.children) > 1):
                c = pathContainer.children
                c = c[0:-1]
                pathContainer.children = c
                pathContainer.children[-1].disabled = False

        addButton.on_click(addPathWidget)
        removeButton.on_click(removePathWidget)

        addPathWidget()

        return fullPathContainer
    
#     def get_data_better(self, prettyPath, options=None):
#         """
#             Does get_data except with the pretty path itself, which should be more robust
        
#             Parameters
#             ----------
            
#             prettyPath: dict
#                 the path
                
#             options: dict
#                 just look at the other docs for this
#         """
        
    def get_data(self, options=None):
        """
           Aggregates and returns data
           
           Flattens out the heirarchal data by combining like data into points using their least common ancestor
           
           Parameters
           ----------
           options: dict
               Options for the aggregation
               
               "format": "list" | "points"
                   the format of the data to be returned
                   list returns a dict of labelled lists of data
                   points returns a list of labelled dict data points
                   default: "points"
                   
            Returns
            -------
            list or dict
                (see options > "format")
        """
        
        def combinePaths(paths):
        #     print(paths)
            if(len(paths) == 1 and len(paths[0]) == 0):
                return None
            possibleIndices = {}
            for path in paths:
                currIndex = path[0]
                try:
                    if int(currIndex) < 0:
                        currIndex = -1
                except:
                    pass
                finally:
                    try:
                        possibleIndices[currIndex].append(path[1:len(path)])
                    except:
                        possibleIndices[currIndex] = [path[1:len(path)]]
            if len(possibleIndices) == 1:
                for key, value in possibleIndices.items():
                    return {key:combinePaths(value)}
            else:
                pathsDict = {}
                for key, value in possibleIndices.items():
                    pathsDict[key] = combinePaths(value)
                return pathsDict
        def getDataPointsHelper(data, combinedPaths, name=""):
        
            def parse(val):
            # only parse strings
                if isinstance(val, str):
                    try:
                        if "int" in self.options["parse"]:
                            return int(val)
                        else:
                            raise Exception()
                    except:
                        try:
                            if "float" in self.options["parse"]:
                                return float(val)
                            else:
                                raise Exception()
                        except:
                            try:
                                if "date" in self.options["parse"]:
                                    return dateParser.parse(val)
                                else:
                                    raise Exception()
                            except:
    #                             traceback.print_exc()
                                return val
                else:
                    return val
    
            returnArray = []
            for key, values in combinedPaths.items():
                if isinstance(data, list):
                    try:
                        if(int(key) >= 0):
                            # somewhat sketchy code here basically is basically just forcing the except to run
                            raise Exception()
                        try:
                            for d in data:
                                try:
                                    returnArray += getDataPointsHelper(d, values, name)
                                except:
                                    pass
                            # if were here, theres only one index, so just return it
                            return returnArray
#                             print(len(returnArray))
                        except:
                            traceback.print_exc()
                    except:
                        # if at this point, it should be a string index
                        for item in data:
                            try:
                                if item['field'] == key:
                                    returnArray += getDataPointsHelper(item, values, name + "/" + key)
                                else:
                                    raise Exception()
                            except:
                                try:
                                    if item['data_fields']['name'] == key:
                                        returnArray += getDataPointsHelper(item, values, name + "/" + key)
                                    else:
                                        raise Exception()
                                except:
                                    pass
                        
                elif isinstance(data, dict):
                    try:
                        if values == None:
                            returnArray += [{name + "/" + key:parse(data[key])}]
                        else:
                            try:
            #                                 print(str(key) + " " + str(values))
                                returnArray += getDataPointsHelper(data[key], values, name + "/" + key)
                            except:
            #                                 returnArray.append(dataDataPointsHelper)
#                                 returnArray.append({key: None})
                                pass
                    except:
            #                         traceback.print_exc()
                        returnArray.append({name + "/" + key: None})
                
            return combineData(returnArray)      

        def combineData(data):
            for i in range(len(data)):
                if data[i] == None:
                    data[i] = {"None":None}
                if not isinstance(data[i], list):
        #             print("in")
                    data[i] = [data[i]]
        #     print("data:")
#             print(data)
            return cartesianProductMany(data)


        def cartesianProductMany(sets):
            a = sets[0]
            for i in range(1,len(sets)):
                a = cartesianProduct(a, sets[i])
            return a

        def cartesianProduct(a, b):
        #     print(a)
        #     print(b)
            points = []
            for i in a:
                for j in b:
                    c = {}
                    c.update(i)
                    c.update(j)
                    points.append(c)
            return points
        
        defaults = {
            "format": "points"
        }
        if isinstance(options, dict):
            for key, value in options.items():
                defaults[key] = value
        
        if not self.formattedData == None:
            if defaults["format"] == "list":
                if isinstance(self.formattedData, dict):
                    return self.formattedData
            else: #format is the default, points
                if isinstance(self.formattedData, list):
                    return self.formattedData
        
        pathsArray = []
        for key, path in self.paths.items():
            pathsArray.append(path)
            
        combinedPaths = combinePaths(pathsArray)
        
        dataPoints = getDataPointsHelper(self.data, combinedPaths)
        
        for index, point in enumerate(dataPoints):
            for name, rule in self.columns.items():
                try:
                    dataPoints[index][name] = eval(rule)
                except:
                    pass
                    
        
        if defaults["format"] == "list":
            dataArrays = {}
            # first, get all the 
            for point in dataPoints:
                for name, value in point.items():
                    if not value == None:
                        dataArrays[name] = []
            
            for point in dataPoints:
                try:
                    for name, _ in dataArrays.items():
                        if point[name] == None:
                            raise Exception()
                            
                    for name, _ in dataArrays.items():
                        dataArrays[name].append(point[name])
                except:
                    pass
            self.formattedData = dataArrays
        else:
            self.formattedData = dataPoints
            
        return self.formattedData
    def set_option(self, key, value):
        """
            Sets an option
            
            Parameters
            ----------
            key: string
                the key (or name) of the object to be set
                
            value: Any
                the value to be attributed to the key
                
            Return
            ------
            None
        """
        self.options[key] = value
        
    def set_options(self, options):
        """
            Sets multiple options
            
            A function to set multiple options
            
            Parameters
            ----------
            options: dict
                a dict of options in form "key":"value" to be set
        """
        
        
        if isinstance(options, dict):
            self.options.update(options)
            
                
    def add_column(self, name, exp, subs):
        """
            Adds a new calculated column
            
            Parameters
            ----------
            exp: str
                the expression to caluate the column
                
            subs: dict
                substitutions to be made in the expression
                ex: {"x":"experiment/dian:cdrsuppdata/data_feilds/date"}
                
            Returns
            -------
            columns: dict
        """
        
        rule = exp
        
        for var, sub in subs.items():
            rule = rule.replace(var, 'point["' + sub + '"]')
            
        self.columns[name] = rule
        
        self.formattedData = None
        
        return self.columns
            
        
        
        

In [ ]:
class SubjectData(Data):
    def __init__(self, options=None):
        """
            Initializes the object
            
            Parameters
            ----------
            options:  dict
                a dict of options
                
                "keys": list
                    a list of strings used to determine which properties of objects in arrays can be used to determine a key for that object
                    default: ["field", "names"]
                    
                "id_field": string
                    the the field name of the subject that contains its unique identifier
                    default: "ID" (this is what is used in CNDA)
                    
                "add_data_mode": "merge" | "replace" | "keep" | "append"
                    the mode for adding subject data
                    "merge":
                        if the added subject id matches an existing subject id, merge the two together
                    "replace":
                        if the added subject id matches an existing subject id, replace the old with the new
                    "keep":
                        if the added subject id matches an existing subject id, keep the old and disregard the new
                    "append":
                        just append the subject to the data -- even if there is a matching subject
                    default: "merge"
                        
                "merge_priority": "old" | "new"
                    if there is a conflict when merging subject data together, keep either the old or new data
                    
            Returns
            -------
            SubjectData
                The SubjectData object
        """
        
        self.subjectGroups = []
        self.groupsDisplayContainer = widgets.VBox()
        self.groupTitleDisplayContainer = widgets.VBox()
        
        defaults = {
            "id_field": "ID",
            "add_data_mode": "merge",
            "merge_priority": "new"
        }
        
        if isinstance(options, dict):
            for key, value in defaults.items():
                try:
                    defaults[key] = options[key]
                except:
                    pass
                
        Data.__init__(self, [], defaults)
        
    def __str__(self):
        return ("SubjectData (" + str(len(self.data)) + " subjects)")
        
    def add_subject(self, subject, options=None):
        """
            Adds new subject to the data
            
            Parameters
            ----------
            subject: dict
                a dict object representing the subject and its contained data
            
            options: dict
                optionally provide options that will be applied only to this operation
                "keys": list
                    a list of strings used to determine which properties of objects in arrays can be used to determine a key for that object
                    default: ["field", "names"]
                    
                "id_field": string
                    the the field name of the subject that contains its unique identifier
                    default: "ID" (this is what is used in CNDA)
                    
                "add_data_mode": "merge" | "replace" | "keep" | "append"
                    the mode for adding subject data
                    "merge":
                        if the added subject id matches an existing subject id, merge the two together
                    "replace":
                        if the added subject id matches an existing subject id, replace the old with the new
                    "keep":
                        if the added subject id matches an existing subject id, keep the old and disregard the new
                    "append":
                        just append the subject to the data -- even if there is a matching subject
                    default: "merge"
                        
                "merge_priority": "old" | "new"
                    if there is a conflict when merging subject data together, keep either the old or new data
            Return
            ------
            None
        """
        
        localOptions = {}
        localOptions.update(self.options)
        if isinstance(options, dict):
            for key, value in options.items():
                localOptions[key] = value
        
        
        if not isinstance(subject, dict):
            raise Exception("Subject must be of type dict")
        
        mode = localOptions["add_data_mode"]

        if mode == "append":
            self.data.append(subject)
        else:
            try:
                newSubjectId = subject[localOptions["id_field"]]
            except:
                raise Exception("Subject does not contain required id field parameter '" + localOptions['id_field'] + "'")
            else:
                if mode in {"replace", "keep"}:
                    for i in range(len(self.data)):
                        oldSubject = self.data[i]
                        oldSubjectId = oldSubject[self.options["id_field"]]
                        if newSubjectId == oldSubjectId:
                            if mode == "replace":
                                self.data[i] = subject
                            return
                    #subject not in array
                    self.data.append(subject)
                    return
                else:
                    #default case: merge
                    mergePriority = localOptions["merge_priority"]
                    for i in range(len(self.data)):
                        oldSubject = self.data[i]
                        oldSubjectId = oldSubject[self.options["id_field"]]
                        if newSubjectId == oldSubjectId:
                            def merge(new, old):
                                for key, value in new.items():
#                                     print(key)
                                    try:
                                        oldValue = old[key]
                                    except:
                                        # the new key didn't previously exist, so add it
                                        old[key] = value
                                    else:
                                        if isinstance(value, list) and isinstance(oldValue, list):
                                            old[key] = oldValue + value
                                        elif isinstance(value, dict) and isinstance(oldValue, dict):
                                            old[key] = merge(value, oldValue)
                                        else:
                                            if not mergePriority == "old":
                                                old[key] = value
                                return old
                                
                            self.data[i] = merge(subject, oldSubject)
                            return
                            
                    #if the subject isn't in the array
                    self.data.append(subject)
                    return    
            

    def add_subjects(self, subjects, options=None):
        """
            Adds an array of new subject to the dat
            
            This function is a simple wrapper that adds subjects to the add by delegating to add_subject()
            
            Parameters
            ----------
            subjects: list
                a lsit of dict objects representing the subject and its contained data
            
            options: dict
                optionally provide options that will be applied only to this operation
                "keys": list
                    a list of strings used to determine which properties of objects in arrays can be used to determine a key for that object
                    default: ["field", "names"]
                    
                "id_field": string
                    the the field name of the subject that contains its unique identifier
                    default: "ID" (this is what is used in CNDA)
                    
                "add_data_mode": "merge" | "replace" | "keep" | "append"
                    the mode for adding subject data
                    "merge":
                        if the added subject id matches an existing subject id, merge the two together
                    "replace":
                        if the added subject id matches an existing subject id, replace the old with the new
                    "keep":
                        if the added subject id matches an existing subject id, keep the old and disregard the new
                    "append":
                        just append the subject to the data -- even if there is a matching subject
                    default: "merge"
                        
                "merge_priority": "old" | "new"
                    if there is a conflict when merging subject data together, keep either the old or new data
            Return
            ------
            None
        """
        for subject in subjects:
            self.add_subjects(subject, options)
    
    def add_subjects_group(self, subjects, title=None):
        """
            Creates a new group from a list of subject ids
            
            Parameters
            ----------
            
            subjects: list
                a list of subject ids
                
            title: string (optional)
                a title for the group.
                if not given, the title will be "Group {number}"
                
            Returns
            -------
             list [[string:Any]]
                A list of all subject groups
        """
        filterGroups = {self.options["id_field"]:subjects}
        return self.add_group()
    
    def add_group(self, filterGroups, title=None):
        """
            Adds a new subject groups
            
            Parameters
            ----------
            
            filterGroups: dict [string:[Any]]
                a dict where the key is the name of the parameter and the value is a list of acceptable values
                for a subject to be in this group, it must satisfy all of the parameters
                ex:
                {
                    "gender":["Male", "Other"]
                }
            
            title: string (optional)
                a title for the group.
                if not given, the title will be "Group {number}"
                
            Returns
            -------
            list [[string:Any]]
                A list of all subject groups
        """
        
        def filterGroupsDescription(filterGroups, delim="; "):
            string = ""
            count = 0
            for key, values in filterGroups.items():
                count += 1
                string += key + ": "
                for i in range(len(values)):
                    if not i == 0:
                        string += ", "
                    string += values[i]
                if not count == len(filterGroups):
                    string += delim
            return string
        
        if title == None:
            title = "Group " + str(len(self.subjectGroups))
            
        newSubjectGroup = {"title":title, "filterGroups":filterGroups, "subjects":SubjectData.subjects_in_filter_groups(self.data, filterGroups)}
        self.subjectGroups.append(newSubjectGroup)
        
        

        newGroupLabel = widgets.HTML(value="<b>" + newSubjectGroup["title"] + ":</b><br>&emsp;" +
                                           str(len(newSubjectGroup["subjects"])) + " subjects" +
                                           ("<br>&emsp;" + filterGroupsDescription(newSubjectGroup["filterGroups"], delim="<br>&emsp;")
                                           if len(newSubjectGroup["filterGroups"]) > 0
                                           else ""))

        groupTitleDisplayContainerChildren = self.groupTitleDisplayContainer.children
        groupTitleDisplayContainerChildren += (newGroupLabel,)
        self.groupTitleDisplayContainer.children = groupTitleDisplayContainerChildren



        checkboxes = []

        #TODO:: Minor optimization, but this doesn't have to regenerate every single checkbox, it only has to add the new one
        for group in self.subjectGroups:

            title = group["title"] + " (" + str(len(group["subjects"])) + " subjects" + ("; " + filterGroupsDescription(group["filterGroups"]) if len(group["filterGroups"]) > 0 else "")  + ")"

            checkbox = widgets.Checkbox(description = title,
                                        value=True,
                                        disabled=False,
                                        layout=widgets.Layout(width='97%', height='40px'))
            checkboxes.append(checkbox)

        if(self.groupsDisplayContainer == None):
            self.groupsDisplayContainer = widgets.VBox()

        self.groupsDisplayContainer.children = checkboxes
        
        return self.subjectGroups
    
    
    def subjects_in_filter_groups(subjects, filterGroups):
        """
            Static method that returns a list of subjects (not just id's) that satisfy the filterGroups
            
            Parameters
            ----------
            
            subjects: list [[string:Any]]
                a list of subjects to be filtered from
                
            filterGroups: dict [string:[Any]]
                a dict where the key is the name of the parameter and the value is a list of acceptable values
                for a subject to be in this group, it must satisfy all of the parameters
                ex:
                {
                    "gender":["Male", "Other"]
                }
            
        """
        def subjectInKeyValues(subject, key, values):
            if(len(values) > 0):
                for value in values:
                    val = subject[key]
                    if val == "":
                        val = "no value"
                    if val == value:
                        return True
                return False
            else:
                return True

        def subjectInGroups(subject, groups):
            for key, values in groups.items():
                if isinstance(values, list):
                    if not subjectInKeyValues(subject, key, values):
                        return False
                else:
                    return subject[key] == values
            return True

        filteredSubjects = []
        if len(filterGroups) > 0:
            for subject in subjects:
                if subjectInGroups(subject, filterGroups):
                    filteredSubjects.append(subject)

        return filteredSubjects
    
    def calculate_groups(self):
        """
            Returns the UI for creating new groups
            
            Returns
            -------
            widgets.VBox
                a widget containing the UI
        """
        def getGroupContainer(group):
            groupContainer = widgets.VBox()
            children = []
            for key, value in group.items():
                checkbox = widgets.Checkbox(description=key + " (" + str(value) + ")",
                                            value=False,
                                            disabled=False,
                                           )
                children.append(checkbox)
            ignoreCheckbox = widgets.Checkbox(description="Ignore these parameters", value=False, disabled=False)
            children.append(ignoreCheckbox)

            groupContainer.children = children

            out = widgets.Output()
            with out:
                labels = []
                values = []
                for key, value in group.items():
                    labels.append(key)
                    values.append(value)

                plt.pie(x=values, labels=labels)
                plt.show()

            return widgets.HBox(children=[groupContainer, out])

        def getGroupsContainer(groups):
            groupsContainer = widgets.Accordion()
            children = []
            for key, value in groups.items():
                groupContainer = getGroupContainer(value)
                groupsContainer.set_title(len(children), key)
                children.append(groupContainer)

            groupsContainer.children = children
            return groupsContainer
        
        groups = {}
        toDelete = []
        for subject in self.data:
            for field, group in subject.items():
                if not (field == "ID" or field == "label" or field == "URI" or isinstance(group, list) or isinstance(group, dict)):
                    if (group == ""):
                        group = "no value"
                    try:
                        num = float(group)
#                         print(field + ": " + group)
                    except:
                        try:
                            groups[field][group] += 1
                        except:
                            try:
                                groups[field][group] = 1
                            except:
                                try:
                                    groups[field] = {}
                                    groups[field][group] = 1
                                except:
                                    pass

        for fieldId, field in groups.items():
            keyCount = 0
            count = 0
            for group, value in field.items():
                if not group == "no value":
                    keyCount += 1
                    count += value

            if 2*keyCount > count or keyCount <= 1:
                toDelete.append(fieldId)

        for fieldId in toDelete:
            del groups[fieldId]

#         print(groups)

        filteredSubjects = {}

        groupsContainer = getGroupsContainer(groups)
        recalculateButton = widgets.Button(description="Create New Group")
        groupTitleText = widgets.Text(value="Group " + str(len(self.subjectGroups)))
        recalculateContainer = widgets.HBox(children=[groupTitleText, recalculateButton])

#         self.groupTitleDisplayContainer = widgets.VBox()

        def recalculateGroups(event):
            filterGroups = {} # field: [allowedGroup1, allowedGroup2]
            for i in range(len(groups)):
                if not groupsContainer.children[i].children[0].children[-1].value:
                    potentialKeys = []

                    for checkbox in groupsContainer.children[i].children[0].children[0:-1]:
                        if not checkbox.description == "Ignore":
                            if(checkbox.value):
                                keyEndIndex = checkbox.description.index(" (")
                                key = checkbox.description[0:keyEndIndex]

                                potentialKeys.append(key)
                    if len(potentialKeys) > 0:
                        filterGroups[groupsContainer._titles[str(i)]] = potentialKeys

            filteredSubjects = SubjectData.subjects_in_filter_groups(self.data, filterGroups) 
#             newSubjectGroup = {"title": groupTitleText.value, "filterGroups": filterGroups, "subjects": filteredSubjects}
            
            self.add_group(filterGroups, groupTitleText.value)
            groupTitleText.value = "Group " + str(len(self.subjectGroups))
        
            
        recalculateButton.on_click(recalculateGroups)
        self.groups_ui = widgets.VBox(children=[groupsContainer, recalculateContainer, self.groupTitleDisplayContainer])
        return self.groups_ui
    
    
    def get_selected_subject_ids(self):
        """
            Gets the subject ids that have been selected through groups
            
            Returns
            -------
            list [[string:Any]]
        """
        subjectIds = []
        for i in range(len(self.groupsDisplayContainer.children)):
            checkbox = self.groupsDisplayContainer.children[i]
            if checkbox.value == True:
                for subject in self.subjectGroups[i]["subjects"]:
                    subjectId = subject[self.options["id_field"]]
                    if not subjectId in subjectIds:
                        subjectIds.append(subjectId)

        return subjectIds
    
    def get_selected_groups_ui(self):
        return self.groupsDisplayContainer

        
        
            
            
            
            
            

In [ ]:
import ipywidgets as widgets
import requests
import getpass

class IncorrectLoginException(Exception):
        pass
    
class XnatUI:
    def __init__(self, server, options=None):
        self.JESSIONID = None
        self.projects = []
        self.projectIds = []
        self.availableProjectsUI = None
        self.fetchedProjectIds = []
        self.subjectData = None
        self.selectedProjectIds = []
        self.experimentTypes = None
        
        if server[-1] == "/":
            self.server = server[0:-1]
        else:
            self.server = server
        defaults = {
            "force_ssl":True
        }
        
        if isinstance(options, dict):
            defaults.update(options)
        self.options = defaults
      
    def __str__(self):
        print("XNAT instance running at " + self.server)
    
    
    def get_login(self, username, password):
        secure = self.options["force_ssl"]
        response = requests.post(self.server + "/data/JSESSIONID",
                                 auth=requests.auth.HTTPBasicAuth(username, password),
                                 verify=secure)
        
#         print(vars(response))
        if(response.status_code == 200):
            cookieString = response.headers["Set-Cookie"]
            indexFirst = cookieString.index("JSESSIONID=")+11
            indexLast = cookieString.index(";", indexFirst)
            self.JSESSIONID = cookieString[indexFirst:indexLast]
            return self.JSESSIONID
        else:
            raise IncorrectLoginException("Username/Password combination incorrect")
            
    def display_login_ui(self):
        username = input("Username: ")
        password = getpass.getpass("Password: ")
        
        try:
            return self.get_login(username, password)
            print("Success")
        except IncorrectLoginException:
#             traceback.print_exc()
            # if login fails, try again
            return self.display_login_ui()
    
    
    
    def get_available_projects(self):
        if (not self.projectIds == None) and len(self.projectIds) > 0:
            return self.projectIds
        
        if self.JSESSIONID == None:
            raise Exception("Must login first before calling this method (call display_login_ui() or get_login(username, password))")
        projectsResponse = requests.get(self.server + "/data/projects",
                                        headers={"Cookie":"JSESSIONID="+self.JSESSIONID},
                                        verify=self.options["force_ssl"])
        try:
            assert projectsResponse.status_code == 200
            projects = projectsResponse.json()["ResultSet"]["Result"]
        except:
            raise Exception("API returned error")
        else:
            projectIds = []
            
            for project in projects:
                projectIds.append(project["ID"])
                
            self.projects = projects
            self.projectIds = sorted(projectIds, key = lambda s: s.lower())
            return self.projectIds
        
    def get_available_projects_ui(self):
        
        #possible improvement: Able to add through regex
        if self.availableProjectsUI == None:
            projectIds = self.get_available_projects()
            self.selectedProjectIds = []

            self.availableProjectsUI = widgets.VBox()
            addedContainer = widgets.VBox()
            addContainer = widgets.HBox()

            self.availableProjectsUI.children = (addedContainer, addContainer)

            dd = widgets.Dropdown(options=projectIds)
            addButton = widgets.Button(description="Add")

            def remove(projectId, b):
                self.availableProjectsUI.children[0].children = tuple(
                                            project
                                            for project in self.availableProjectsUI.children[0].children
                                            if not project.children[1].value == projectId
                                           )
                try:
                    self.selectedProjectIds.remove(projectId)
                except:
                    pass

            def add(b):
                # add to the data
                currentId = dd.value
                if not currentId in self.selectedProjectIds:
                    self.selectedProjectIds.append(currentId)

                    # add to the UI

                    title = widgets.HTML(value=currentId)
                    button = widgets.Button(description="Remove")

                    button.on_click(lambda b: remove(currentId, b))

                    childrenArray = self.availableProjectsUI.children[0].children
                    childrenArray = childrenArray + (widgets.HBox(children=[
                        button, title
                    ]),)

                    self.availableProjectsUI.children[0].children = childrenArray


            addButton.on_click(add)
            addContainer.children = (dd, addButton)

            return self.availableProjectsUI
        else:
            return self.availableProjectsUI
        
    def get_selected_projects(self):
        
        for project in self.projects:
            if project["ID"] in self.selectedProjectIds:
                selectedProjects.append(project)
                
        return selectedProjects
    
    def select_project(self, project):
        self.selectedProjectIds.append(project)
        
        return self.selectedProjectIds
    
    def select_projects(self, projects):
        self.selectedProjectIds += projects
        
        return self.selectedProjectIds
    
    def fetch_subjects(self):
        projectsToGet = [project for project in self.selectedProjectIds if not project in self.fetchedProjectIds]
        requests = (grequests.get(self.server + "/data/projects/" + project + "/subjects",
                                   headers = {"Cookie":"JSESSIONID="+self.JSESSIONID},
                                   params = {
                                       "format":"json",
                                       "columns":"""age,birth_weight,dob,education,educationDesc,
                                       ethnicity,gender,gestational_age,group,handedness,
                                       height,insert_date,insert_user,last_modified,pi_firstname,
                                       pi_lastname,post_menstrual_age,race,ses,src,weight,yob"""
                                   },
                                   verify = self.options["force_ssl"]) for project in projectsToGet)
            
        results = grequests.map(requests)
        
        if self.subjectData == None:
            self.subjectData = SubjectData({"id_field":"ID"})
        
        for count, result in enumerate(results):
            if result.status_code == 200:
                try:
                    subjects = result.json()["ResultSet"]["Result"]
                except:
                    traceback.print_exc()
                    pass
                else:
                    self.fetchedProjectIds.append(projectsToGet[count])
                    for i in range(len(subjects)):
                        subjects[i]["project"] = projectsToGet[count]
                        subjects[i]["experiments"] = {}
                        self.subjectData.add_subject(subjects[i])
                        
        return self.subjectData
    
    def get_available_experiments(self):
        experimentsRequests = (grequests.get(self.server + "/data/projects/" + project + "/experiments",
                               headers = {"Cookie":"JSESSIONID="+self.JSESSIONID},
                               params = {
                                   "format":"json",
                                   "columns":"xsiType,label,subject_ID,ID"
                               },
                               verify = self.options["force_ssl"]) for project in self.selectedProjectIds)
        experimentsResults = grequests.map(experimentsRequests)
        
        groupIds = {}    
        expGroupCount = {}
        experimentTypes = {}
        
        
        
        for result in experimentsResults:
            
            
            
            try:

                exp = result.json()["ResultSet"]["Result"]
            except:
                print("You are not logged in")
            else:
                for e in exp:
                    try:
                        experimentTypes[e["xsiType"]][e["subject_ID"]].append(e["ID"])
                    except:
                        try:
                            experimentTypes[e["xsiType"]][e["subject_ID"]] = [e["ID"]]
                        except:
                            experimentTypes[e["xsiType"]] = {
                                e["subject_ID"]: [e["ID"]]
                            }
#                     for group, subs in groupIds.items():
#                         subId = e["subject_ID"]
#                         expType = e["xsiType"]
#                         if subId in subs:
#                             try:
#                                 expGroupCount[expType][group] += 1
#                             except:
#                                 try:
#                                     expGroupCount[expType][group] = 1
#                                 except:
#                                     expGroupCount[expType] = {group: 1}
                
        self.experimentTypes = experimentTypes
        return self.experimentTypes
    def get_available_experiments_ui(self):
        
        if self.experimentTypes == None or len(self.experimentTypes) == 0:
            self.get_available_experiments()
            
        groupIds = {}
        for subjectGroup in self.subjectData.subjectGroups:
            groupIds[subjectGroup["title"]] = []
            for subject in subjectGroup["subjects"]:
                groupIds[subjectGroup["title"]].append(subject["ID"])
                
        
        expGroupCount = {}
        
        for expId, subjects in self.experimentTypes.items():
            expGroupCount[expId] = {}
            for groupId, subjectIds in groupIds.items():
                expGroupCount[expId][groupId] = 0;
                for subjectId in subjectIds:
                    try:
                        count = len(self.experimentTypes[expId][subjectId])
                    except:
                        pass
                    else:
                        expGroupCount[expId][groupId] += count
                    
                
                
                
        options = []
        for xsi, subjectId in self.experimentTypes.items():
            title = xsi + " ("
            for subjectGroup in self.subjectData.subjectGroups:
                count = 0
                group = subjectGroup["title"]
                try:
                    count = expGroupCount[xsi][group]
                except:
                    pass
                title += str(group) + ": " + str(count) + " experiments; "
            options.append(title[0:-2] + ")")
            
        experimentButtons = widgets.Dropdown(options=sorted(options),
                          
                       description="Data Types: ",
                                                 disabled=False,
                                                 value=None)
        self.experimentContainer = widgets.HBox()
        experimentGoButton = widgets.Button(description='Go')
        experimentGoButton.on_click(self.get_experiment_from_ui)
        
        self.experimentContainer.children = [experimentButtons, experimentGoButton]
        
        return self.experimentContainer
    
    def get_experiment_from_ui(self, event=None):
        codeStr = self.experimentContainer.children[0].value
        endIndex = codeStr.index(" (")
        experimentCode = codeStr[0:endIndex]
        return self.get_experiment(experimentCode, self.subjectData.get_selected_subject_ids())
        
        
    def get_experiment(self, experimentCode, subjectIds=None):
        experiments = []
        experimentSubjects = self.experimentTypes[experimentCode]
        if subjectIds == None:
            subjectIds = self.subjectData.get_selected_subject_ids()
        for subject in subjectIds:
            try:
                experiments += experimentSubjects[subject]
            except:
                pass

        requests = [grequests.get(self.server + "/data/experiments/" + exp,
                                 headers = {"Cookie":"JSESSIONID="+self.JSESSIONID},
                                 params = {"format":"json"},
                                 verify = self.options["force_ssl"]) for exp in experiments]

        def exceptionHandler(req, exc):
            print(exc)

        result = grequests.map(requests, exception_handler=exceptionHandler)

    #     print("woo")
    #     print(result)
    #     print(result[0]._content)


        for exResult in result:
            try:
                exDict = json.loads(exResult._content.decode('utf-8'))['items'][0]
            except:
                pass    
            else:
                exSubjectId = exDict['data_fields']['subject_ID']
                self.subjectData.add_subject({"ID":exSubjectId, "experiments":{exDict['meta']['xsi:type']:[exDict]}},
                                        options={'add_data_mode':'merge', 'merge_priority':'old'})
                
                
        print("Done getting experiments")
        
        
# temp      
# xnat = XnatUI("https://cnda-dev-aidan1.nrg.mir", {"force_ssl":False})
# # xnat.display_login_ui(); # ';' will suppress output
# xnat.get_login('aidan','')
# # xnat.get_available_projects_ui()
# xnat.select_project('DIAN_011')
# xnat.fetch_subjects().calculate_groups()
# xnat.fetch_subjects().add_group({'ID':'CNDA0516'}, "CNDA0516")
# xnat.get_available_experiments_ui()
# # xnat.subjectData.groupsDisplayContainer
# # subjects = xnat.subjectData.add_group({'insert_user':['dan']})[-1]["subjects"]
# # subjectIds = []
# # for subject in subjects:
# #     subjectIds.append(subject["ID"])
# # print(subjectIds)
# # xnat.get_available_experiments()
# xnat.get_available_experiments_ui()



In [ ]:
# xnat.subjectData.get_gui()
xnat.subjectData.get_data({"format":"list"})

In [ ]:
xnat.fetch_subjects().calculate_groups()

In [ ]:
# xnat.get_login('aidan','')
XnatUI.get_available_experiments_ui(xnat)

In [ ]:
xnat.subjectData.get_gui()


In [ ]:
xnat.subjectData.get_data({"format":"points"})

In [ ]:
# data = Data({"b":[{"a":{"g":"x", "h":4}}, {"a":{"g":"h"}}], "id":7})
# display(data.get_gui())

sub = SubjectData(options={
    "add_data_mode":'merge',
    "merge_priority":"new"
})
sub.add_subject({
                 "ID":"1",
                 "a parameter":4,
                 "exp":{
                     "a":["first type a experiment"]
                    },
                "children": [
                    {
                        "field":"test",
                        "param":'8',
                        "asdf":7,
                        "eee": [
                            {
                                "field":"i"
                            }
                        ]
                    },
                    {
                        "field":"test2",
                        "param":8,
                        "asdf":7
                    },
                ]
                })
sub.add_subject({
                 "ID":2,
                 "a parameter": 9,
                 "exp":{
                     "b":["a type b experiment"],
                     "a":["second type a experiment"]
                     },
                     "children": [
                    {
                        "field":"test2",
                        "param":"2018-08-29",
                        "asdf":3
                    },
                    {
                        "field":"test",
                        "eee": [
                            {
                                "field":"i"
                            }
                        ],
                        "param":8,
                        "asdf":7
                    },
                ],
                 "another parameter":7
                })

sub.data

sub.get_gui()

In [ ]:
Data.get_data(sub, {"format":"points"})

In [ ]:
sub.add_column("age", "x - y", {"x":"test/asdf", "y":"test2/asdf"})

In [ ]:
xnat = XnatUI("https://cnda-dev-aidan1.nrg.mir", {"force_ssl":False})
# xnat.display_login_ui(); # ';' will suppress output
xnat.get_login('aidan','')

In [ ]:
xnat.get_available_projects_ui()

In [ ]:
xnat.fetch_subjects()

In [ ]:
SubjectData.calculate_groups(xnat.subjectData)

# xnat-io

In [ ]:
xnat = XnatUI("https://cnda-dev-aidan1.nrg.mir", {"force_ssl":False})
# xnat.display_login_ui(); # ';' will suppress output

In [ ]:
xnat.get_login("aidan", "");

In [ ]:
xnat.get_available_projects_ui()

In [ ]:
xnat.fetch_subjects().calculate_groups()

In [ ]:
xnat.subjectData.groupsDisplayContainer

In [ ]:
xnat.get_available_experiments_ui()

In [ ]:
SubjectData.get_gui(XnatUI.fetch_subjects(xnat))

In [ ]:
xnat.fetch_subjects().data[0]['experiments']['dian:cdrsuppData'][-1]

In [ ]:
xnat.fetch_subjects().add_column('age', 'date - dob', {'date':'/experiments/dian:cdrsuppData/data_fields/date', 'dob':'/dob'})


In [ ]:
print(xnat.fetch_subjects().paths)
data = Data.get_data(xnat.fetch_subjects(), {"format":"p"})
data

In [ ]:
# calculating a new column -- will be a future feature
# for i in range(len(data)):
#     try:
#         # parsing dates -- will be done automatically in the future
#         dob = dateParser.parse(data[i]["dob"])
#         currentDate = dateParser.parse(data[i]["date"])
#     except:
#         data[i]["age"] = None
#     else:
#         data[i]["age"] = (currentDate - dob).days / 365.2425
        
ageList = []
judgmentList = []

# turning points into lists -- already supported, but not for calculated columns
for point in data:
    try:
        age = float(point["age"])
        judgment = float(point["JUDGMENT"])
    except:
        pass
    else:
        ageList.append(age)
        judgmentList.append(judgment)
        
plt.scatter(ageList, judgmentList);
plt.xlabel("age");
plt.ylabel("judgement");

In [ ]:
from matplotlib import pyplot as plt

ageListTu = []
ageListDp = []

for point in data:
    try:
        age = float(point["age"])
    except:
        pass
    else:
        if point["group"] == "tu":
            ageListTu.append(age)
        else:
            ageListDp.append(age)


%matplotlib
bins = [range(0, 100, 6)]
plt.hist(ageListTu, alpha=0.5, label='tu', density=True);
plt.hist(ageListDp, alpha=0.5, label='dp', density=True);
plt.legend();
plt.xlabel("subject age");
plt.ylabel("proportion");